In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
from IPython.display import clear_output
sns.set()

def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

In [2]:
device = torch.device('cuda:0')
device

device(type='cuda', index=0)

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


#### Data pathes

If you are using Google colab left it as it is. 

Otherwise, if you are running notebook locally, change pathes accordinly.

In [0]:
train_data_path = '/gdrive/My Drive/mlhep2019/data_train.npz'
val_data_path = '/gdrive/My Drive/mlhep2019/data_val.npz'
test_data_path = '/gdrive/My Drive/mlhep2019/data_test.npz'

# Loading data

Data is stored in `.npz`-format which is a special filetype for persisting multiple NumPy arrays on disk. 

More info: https://docs.scipy.org/doc/numpy/reference/generated/numpy.lib.format.html#module-numpy.lib.format.

File `dat_train.npz` contains four arrays: 

  * `EnergyDeposit` - images of calorimeters responses
  * `ParticleMomentum` - $p_x, p_y, p_z$ of initial partice
  * `ParticlePoint` - $x, y$ of initial particle
  * `ParticlePDG` - particle type(either $e^-$ or $\gamma$)

In [5]:
# open train dataset
data_real = np.load(train_data_path, allow_pickle=True)
print(list(data_real.keys()))

# [data_size, 900]
EnergyDeposit = data_real['EnergyDeposit']
# reshaping it as [data_size, channels, img_size_x, img_size_y]
# channels are needed for pytorch conv2d-layers
EnergyDeposit = EnergyDeposit.reshape(-1, 1, 30, 30)

# [data_size, 3]
ParticleMomentum = data_real['ParticleMomentum']

# [data_size, 2]
ParticlePoint = data_real['ParticlePoint']

# [data_size, 1]
ParticlePDG = data_real['ParticlePDG']

# augment training dataset
# absolute momentum
ParticleAbsMomentum2 = (np.power(data_real['ParticleMomentum'][:,0], 2) + 
                        np.power(data_real['ParticleMomentum'][:,1], 2) + 
                        np.power(data_real['ParticleMomentum'][:,2], 2))
ParticleAbsMomentum = np.power(ParticleAbsMomentum2, 0.5)

# particle energy
m_e2 = np.ones_like(ParticlePDG) * (0.000511**2)
ParticleEnergy_e = (data_real['ParticlePDG'] == 11) * ParticleAbsMomentum
ParticleEnergy_gamma = (data_real['ParticlePDG'] == 22) * (np.sqrt(ParticleAbsMomentum2 + m_e2))
ParticleEnergy = ParticleEnergy_gamma + ParticleEnergy_e



['EnergyDeposit', 'ParticlePoint', 'ParticleMomentum', 'ParticlePDG']


In [0]:
# not actully interested in p_z momentum
ParticleMomentum = ParticleMomentum[:, :2]
ParticlePoint = ParticlePoint[:, :2]

# Loading it to pytorch `DataLoader`

  1. Convert from `numpy`-array to Torch `tensors`

In [0]:
EnergyDeposit = torch.tensor(EnergyDeposit).float()
ParticleMomentum = torch.tensor(ParticleMomentum).float()
ParticlePoint = torch.tensor(ParticlePoint).float()

In [0]:
AdditionalInfoP = torch.tensor(ParticleAbsMomentum).float()
AdditionalInfoE = torch.tensor(ParticleEnergy).float()

  2. Convert three `tensors` to `TensorDataset`-format
  3. Wrapping it with `DataLoader`

In [0]:
BATCH_SIZE = 1024
calo_dataset = utils.TensorDataset(EnergyDeposit, ParticleMomentum, ParticlePoint)
calo_dataloader = torch.utils.data.DataLoader(calo_dataset, 
                                              batch_size=BATCH_SIZE, 
                                              pin_memory=True, shuffle=True)

In [0]:
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, 2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, 2)
        self.conv4 = nn.Conv2d(64, 64, 2)
                
        self.fc1 = nn.Linear(1600, 512) 
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 16)
        self.fc6 = nn.Linear(16, 2 + 2)
        
        self.dp1 = nn.Dropout(p=0.05)

        
    def forward(self, x):
        x = F.leaky_relu((self.conv1(x)))
        x = F.leaky_relu((self.conv2(x)))
        x = F.leaky_relu((self.conv3(x)))
        x = F.leaky_relu((self.conv4(x))) # 64, 5, 5
        x = x.view(len(x), -1)

        x = F.leaky_relu(self.fc1(x))
        x = self.dp1(x)
        x = F.leaky_relu(self.fc2(x))
        x = F.leaky_relu(self.fc3(x))
        x = F.leaky_relu(self.fc4(x))
        x = F.leaky_relu(self.fc5(x))
        return self.fc6(x)

In [0]:
regressor = Regressor().to(device)

# Defining optimizer


In [0]:
learning_rate = 3e-4
opt = optim.Adam(regressor.parameters(), lr=learning_rate)

## Relative MSE that is used in competition

In [0]:
ParticleMomentum_mean, ParticlePoint_mean = ParticleMomentum.mean(dim=0), ParticlePoint.mean(dim=0)
ParticleMomentum_ParticlePoint_mean = torch.cat([ParticleMomentum_mean, ParticlePoint_mean]).to(device)

def metric_relative_mse(y_true, y_pred):
    return ((y_true - y_pred).pow(2).mean(dim=0) / (y_true - ParticleMomentum_ParticlePoint_mean).pow(2).mean(dim=0)).sum()

# Loss function

In this example we are using `L1Loss`. 

But maybe it's better to stick to another loss?.. `MSE` or `log-cosh`?..

In [0]:
# loss_fn = torch.nn.L1Loss().to(device)
loss_fn = metric_relative_mse

In [0]:
class RunningAverageMeter(object):
    """
    Computes and stores the average and current value
    
    """
    def __init__(self, momentum=0.99):
        self.momentum = momentum
        self.reset()

    def reset(self):
        self.val = None
        self.avg = 0

    def update(self, val):
        if self.val is None:
            self.avg = val
        else:
            self.avg = self.avg * self.momentum + val * (1 - self.momentum)
        self.val = val

In [0]:
def run_training(epochs=100):
    losses = []
    metrics = []
    # init running average
    loss_meter = RunningAverageMeter(momentum=0.99)
    metric_meter = RunningAverageMeter(momentum=0.99)
    # iterating over epochs...
    for epoch in tqdm(range(epochs)):
        # ...and over batches
        for EnergyDeposit_b, ParticleMomentum_b, ParticlePoint_b in calo_dataloader:
            # moving them to device(for example, cuda-device)
            EnergyDeposit_b, ParticleMomentum_b, ParticlePoint_b = EnergyDeposit_b.to(device), \
                                                                   ParticleMomentum_b.to(device), \
                                                                   ParticlePoint_b.to(device)
            
            # predicting an array of size [batch_size, 4]
            pred = regressor(EnergyDeposit_b)

            # calc loss function
            loss = loss_fn(pred, torch.cat([ParticleMomentum_b, ParticlePoint_b], dim=1))

            # manually zeroing gradients from previous step
            opt.zero_grad()
            
            # and calculating new gradients based on value of loss function
            loss.backward()
            
            # updating weights
            opt.step()
            
            # storing metrics for vizualization
            loss_meter.update(loss.item())
            metric_meter.update(metric_relative_mse(pred, torch.cat([ParticleMomentum_b, ParticlePoint_b], dim=1)).item())
            losses.append(loss_meter.avg)
            metrics.append(metric_meter.avg)
    return losses, metrics

In [17]:
# 10 epoch enought?
losses, metrics = run_training(300)

 68%|██████▊   | 204/300 [06:04<02:52,  1.79s/it]

KeyboardInterrupt: ignored

In [18]:
# plot loss and our metric
plt.figure(figsize=(12, 12))
plt.plot(losses, label='Loss')
plt.legend()
plt.show()
        
plt.figure(figsize=(12, 12))
plt.plot(metrics, label='Metric')
plt.legend()
plt.show()

NameError: ignored

<Figure size 864x864 with 0 Axes>

## Make predictions for validation set

In `data_val.npz` and `data_test.npz` you only have one key: `EnergyDeposit`.

In [0]:
# load validation data
data_val = np.load(val_data_path, allow_pickle=True)
EnergyDeposit_val = data_val['EnergyDeposit']
EnergyDeposit_val = EnergyDeposit_val.reshape(-1, 1, 30, 30)

In [0]:
# predicting [data_num, 4] array with px, py, x, y
prediction_val = regressor.cpu()(torch.tensor(EnergyDeposit_val).float())

In [0]:
# splitting ParticleMomentum and ParticlePoint in two arrays
ParticleMomentum_val, ParticlePoint_val = prediction_val.detach().numpy()[:, :2], prediction_val.detach().numpy()[:, 2:]

In [0]:
# saving predictions in .npz format
np.savez_compressed('data_val_prediction.npz', 
                    ParticlePoint=ParticlePoint_val, 
                    ParticleMomentum=ParticleMomentum_val)

## Make predictions for test set

In [0]:
# loading test dataset
data_test = np.load(test_data_path, allow_pickle=True)
EnergyDeposit_test = data_test['EnergyDeposit']
EnergyDeposit_test = EnergyDeposit_test.reshape(-1, 1, 30, 30)

In [0]:
# predicting [data_num, 4] array with px, py, x, y
prediction_test = regressor.cpu()(torch.tensor(EnergyDeposit_test).float())

In [0]:
# splitting ParticleMomentum and ParticlePoint in two arrays
ParticleMomentum_test, ParticlePoint_test = prediction_test.detach().numpy()[:, :2], prediction_test.detach().numpy()[:, 2:]

In [0]:
# saving predictions in .npz format
np.savez_compressed('data_test_prediction.npz', 
                    ParticlePoint=ParticlePoint_test, 
                    ParticleMomentum=ParticleMomentum_test)

## `zip-zip` files together

In [27]:
!zip solution.zip data_val_prediction.npz data_test_prediction.npz

  adding: data_val_prediction.npz (deflated 0%)
  adding: data_test_prediction.npz (deflated 0%)


In [28]:
from IPython.display import FileLink
FileLink('./solution.zip')

/content/solution.zip

In Google Colab you might not be able to download you solution from browser. Then you can download it from left sidebar of Colab:

![](https://github.com/philippgadow/mlhep2019_1_phase/blob/master/analysis/colab_download.png?raw=1)

## Future steps:

1. Tune arcitecture 
  * stack moar layers :)
  * different types on nonlinearities, hyperparameters of `Conv2d`-layer, initializations, etc...
  * dropout & other regularizations
  
  
2. Play with optimization procedure
  * train for more epochs
  * maybe looking at train metricis is not the best way to prevent overfitting :)
  * learning rate scheduler
  * early stopping
  * different types of loss functions: https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0
  * SWA: https://pytorch.org/blog/stochastic-weight-averaging-in-pytorch/
  
  
3. data augmentation
  * rotate & shift images(do not forget to transform $p_x, p_y$, $x, y$ as well!)
  * adding nose to images/target variables
  
  
4. other trick
  * train to predict $p_z$ and particle type: multi-task or/and transfer learning( http://rail.eecs.berkeley.edu/deeprlcourse-fa17/f17docs/lecture_15_multi_task_learning.pdf )
  * normalization of input/output data: Box-Cox transformation